In [1]:
import pandas as pd
import numpy as np
import hashlib
import re
import random as rd
import datetime
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from similarity.levenshtein import Levenshtein
from similarity.normalized_levenshtein import NormalizedLevenshtein

import sys
sys.path.append('../')

from collections import Counter
import matplotlib.pyplot as plt

In [2]:
from importlib import reload
import EmbDI.data_preprocessing as dp

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/spoutnik23/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
reload(dp)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/spoutnik23/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'EmbDI.data_preprocessing' from '../EmbDI/data_preprocessing.py'>

#### Loading the first dataset

In [35]:
f1 = 'pipeline/experiments/itunes_amazon/exp_data/tableA.csv'
df1 = pd.read_csv('../' + f1, encoding='utf-8')
print(df1.shape)
df1.head(5)

(6907, 9)


,id,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
0,0,Runaway Train,Cam,Welcome to Cam Country - EP,"Country , Music , Contemporary Country , Honky...",$ 0.99,2015 Sony Music Entertainment,3:01,31-Mar-15
1,1,Track 14,Omi,Me 4 U,"Pop/Rock , Music , Pop , Dance , R&B / Soul",Album Only,"2015 Ultra Records , LLC under exclusive licen...",3:41,NaN
2,2,Lips Are Movin,Meghan Trainor,Title,"Pop , Music , Pop/Rock , Dance , Rock , Teen Pop",$ 1.29,"2014 , 2015 Epic Records , a division of Sony ...",3:01,9-Jan-15
3,3,I Want a Hippopotamus for Christmas,A Great Big World,I 'll Be Home For Christmas,"Holiday , Music",$ 1.29,"Compilation ( P ) 2014 Epic Records , a divisi...",2:20,24-Nov-14
4,4,Credit,Meghan Trainor,Title ( Deluxe ),"Pop , Music , Rock , Pop/Rock , Dance , Teen Pop",$ 1.29,"2014 , 2015 Epic Records , a division of Sony ...",2:51,9-Jan-15


#### Loading the second dataset

In [14]:
f2 = 'pipeline/experiments/itunes_amazon/exp_data/tableB.csv'
df2 = pd.read_csv('../' + f2, encoding='utf-8')
print(df2.shape)
df2.head(5)

(55923, 9)


,id,Song_Name,Artist_Name,Album_Name,Genre,Price,CopyRight,Time,Released
0,0,Saxophone Stomp [ Explicit ],Rusko,! ( Volume 2 ) [ Explicit ],"Dance & Electronic , Dubstep",$ 1.29,( C ) 2014 FMLY Under Exclusive License To Uni...,3:20,"September 16 , 2014"
1,1,I Wan na Mingle [ feat . Pusher ],Rusko,! ( Volume 2 ) [ Explicit ],"Dance & Electronic , Dubstep",$ 1.29,( C ) 2014 FMLY Under Exclusive License To Uni...,2:36,"September 16 , 2014"
2,2,Lytah,Rusko,! ( Volume 2 ) [ Explicit ],"Dance & Electronic , Dubstep",$ 1.29,( C ) 2014 FMLY Under Exclusive License To Uni...,3:48,"September 16 , 2014"
3,3,Slappy Pappy,Rusko,! ( Volume 2 ) [ Explicit ],"Dance & Electronic , Dubstep",$ 1.29,( C ) 2014 FMLY Under Exclusive License To Uni...,3:55,"September 16 , 2014"
4,4,Fushion Jam,Rusko,! ( Volume 2 ) [ Explicit ],"Dance & Electronic , Dubstep",$ 1.29,( C ) 2014 FMLY Under Exclusive License To Uni...,1:48,"September 16 , 2014"


#### Setting the Data Preprocessing parameters

In [46]:
e = list(df1.columns) + list(df2.columns)
expand_col = ','.join(e)

#### Generating the basic dataset in the heuristic case. 

In [47]:
parameters = {
    'output_file': 'itunes_amazon',
    'concatenate': 'outer',
    'missing_value': 'nan,ukn,none,unknown,',
    'missing_value_strategy': '',
    'round_number': 1,
    'round_columns': '',
    'auto_merge': False,
    'tokenize_shared': True 
}

df_c = dp.data_preprocessing([df1.drop('id', axis=1), df2.drop('id', axis=1)], parameters)
# df_c = df_c.drop('id', axis=1)
df_c.to_csv('../pipeline/datasets/itunes_amazon/itunes_amazon-heuristic-er.csv', index=False)

../EmbDI/data_preprocessing.py:76: UserWarning: No attributes chosen to round.
  warnings.warn('No attributes chosen to round.')


#### Generating the basic dataset in the basic case. 

In [48]:
parameters = {
    'output_file': 'itunes_amazon',
    'concatenate': 'outer',
    'missing_value': 'nan,ukn,none,unknown,',
    'missing_value_strategy': '',
    'round_number': 1,
    'expand_columns': expand_col,
    'auto_merge': False,
    'tokenize_shared': False 
}

df_c = dp.data_preprocessing([df1.drop('id', axis=1), df2.drop('id', axis=1)], parameters)
# df_c = df_c.drop('id', axis=1)
df_c.to_csv('../pipeline/datasets/itunes_amazon/itunes_amazon-basic-er.csv', index=False)

#### Generating the schema matching dataset in the heuristic case. 

In [49]:
parameters = {
    'output_file': 'itunes_amazon',
    'concatenate': 'horizon',
    'missing_value': 'nan,ukn,none,unknown,',
    'missing_value_strategy': '',
    'round_number': 1,
    'round_columns': '',
    'auto_merge': False,
    'tokenize_shared': True 
}

df_c = dp.data_preprocessing([df1.drop('id', axis=1), df2.drop('id', axis=1)], parameters)
# df_c = df_c.drop('id', axis=1)
df_c.to_csv('../pipeline/datasets/itunes_amazon/itunes_amazon-heuristic-sm.csv', index=False)

#### Generating the schema matching dataset in the basic case. 

In [25]:
parameters = {
    'output_file': 'itunes_amazon',
    'concatenate': 'horizon',
    'missing_value': 'nan,ukn,none,unknown,',
    'missing_value_strategy': '',
    'round_number': 1,
    'expand_columns': df1.columns + df2.columns,
    'auto_merge': False,
    'tokenize_shared': False 
}

df_c = dp.data_preprocessing([df1.drop('id', axis=1), df2.drop('id', axis=1)], parameters)
# df_c = df_c.drop('id', axis=1)
df_c.to_csv('../pipeline/datasets/itunes_amazon/itunes_amazon-basic-sm.csv', index=False)

In [17]:
lcol = len(df_c.columns)//2
for _ in range(lcol):
    print('{},{}'.format(df_c.columns[_],df_c.columns[_+lcol]))

0_album_name,1_album_name
0_artist_name,1_artist_name
0_copyright,1_copyright
0_genre,1_genre
0_price,1_price
0_released,1_released
0_song_name,1_song_name
0_time,1_time


In [18]:
dp.write_info_file([df1, df2], 'info-itunes_amazon.txt', [f1,f2])

#### Prepare the match file

In [19]:
import os
import os.path

tot_m = 0
dir_path = '../pipeline/experiments/itunes_amazon/exp_data/'
with open('../pipeline/matches/matches-itunes_amazon.txt', 'w') as fo:
    for file in os.listdir(dir_path):
        if file not in [os.path.basename(_) for _ in [f1, f2]]:        
            print(file)
            m = 0
            with open(dir_path + file, 'r') as fp:
                for idx, line in enumerate(fp):
                    m1, m2, flag = line.rstrip().rsplit(',')
                    if flag == '1':
                        s = 'idx_{0},idx_{1}\n'.format(m1, str(int(m2) + len(df1)))
                        fo.write(s)
                        m+=1
            print('File {}: {} matches.'.format(file, m))
            tot_m+=m
print('Total matches: {}'.format(tot_m))

test.csv
File test.csv: 27 matches.
train.csv
File train.csv: 78 matches.
valid.csv
File valid.csv: 27 matches.
Total matches: 132
